# Chicago DS Summit Workshop: Finetuning GPT models at Scale
 ----

Note this Demo is based on https://github.com/pytorch/vision/tree/v0.11.3

This notebook walks you through finetuning your own chatbot.
We will learn waht are Generative Pretrained Transformers (GPT), how to finetune to your own domain, and finetune at Scale

Overview of Tutorial:

* What is a GPT model and Causal Modeling
* Establish Baseline Chatbot to ask Data Science questions
* Finetune GPT model on a Data Science book to better answer Data Science Tasks
* Establish Baseline Chatbot to convert english to latex
* Finetune GPT model on dataset to convert english to latex
* (TODO) See benefit of pretraining on domain specific text (i.e. Calculus book) to improve English to Latex
* (TODO) User exercises: Download Shakespeare book from Project Gutenburg, and finetune bot to speak like Shakespeare

Let's get started!

---

In [33]:
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, \
                         Trainer, TrainingArguments
import torch
from datasets import Dataset
import pandas as pd

# What is GPT

In [ ]:
#TODO

# Data Science Chatbot

## Baseline
`A hypothesis test is`

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')  # load up a GPT2 model
pretrained_generator = pipeline(
    'text-generation', model=model, tokenizer='gpt2',
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

In [22]:
PROMPT='A test statistic is'

In [23]:

print('----------')
for generated_sequence in pretrained_generator(PROMPT, num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------
A test statistic is a measure of the success rate of a work from each country in getting this measure set up and using it.

I would add that I don't know whether the American people consider this test "common sense," or to what
----------
A test statistic is a fact. A standard deviation (standard deviation) of two values of the standard deviation of a statistic is a valid metric.

So, if you think your normalization statistic is 4%, we can assume the mean deviation is
----------
A test statistic is in the neighborhood of $1.85.

This suggests that, as of June of this year, all schools across California are failing. One reason for this is an ongoing lack of funding for primary education programs. So the
----------


## Finetune Model

In [49]:
!det experiment create \
    /run/determined/workdir/gpt2-determined-finetune-poc/determined_files/const_ds_chatbot.yaml \
    /run/determined/workdir/gpt2-determined-finetune-poc/determined_files/ 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Preparing files to send to master... 19.7KB and 14 files
Created experiment 33


## See Result

In [80]:
!det checkpoint download 38443886-003c-4f28-97ae-108992665643

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Local checkpoint path:
checkpoints/38443886-003c-4f28-97ae-108992665643 

   Experiment ID |   Trial ID |   Steps Completed | Report Time              | Checkpoint UUID                      | Validation Metrics                       | Metadata
-----------------+------------+-------------------+--------------------------+--------------------------------------+------------------------------------------+----------------------------------------
              33 |         33 |               137 | 2023-03-24 03:00:52+0000 | 38443886-003c-4f28-97ae-108992665643 | {                                        | {
                 |            |                   |                 

In [82]:
# !ls ./checkpoints/71dda373-9065-4536-81e0-4acceb7db318/

In [81]:
loaded_model = GPT2LMHeadModel.from_pretrained('gpt2')
ckpt = torch.load('./checkpoints/38443886-003c-4f28-97ae-108992665643/state_dict.pth')
# print(len(ckpt['models_state_dict']))
loaded_model.load_state_dict(ckpt['models_state_dict'][0])

finetuned_generator = pipeline(
    'text-generation', model=loaded_model, tokenizer=tokenizer,
    config={'max_length': 200,  'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

In [85]:
PROMPT='Sinan is'

In [86]:

print('----------')
for generated_sequence in finetuned_generator(PROMPT, num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------
Sinan is not.
We will see what happens. For ease of use and in order to
make this book, let's go ahead and define a few parameters in order to
optimize the data and the presentation in the
future.
----------
Sinan is the best-known and best-known in mathematics. His most recent book, The Mathematics of Series Form, is well known for being considered one of the most important books on the subject of data science. His latest book on the topic
----------
Sinan is working on a very big project. He is trying to do away with a little more machine learning and focus on a much more
complex set of datasets, the Bayesian data analysis.
We may see a few different
samples
----------


# English 2 Latex

## Baseline

In [18]:
# Sanity check that a non-finetuned model could not have done this
MODEL='gpt2'
non_finetuned_latex_generator = pipeline(
    'text-generation', 
    model=GPT2LMHeadModel.from_pretrained(MODEL),  # not fine-tuned!
    tokenizer=tokenizer
)

In [74]:
text_sample = 'f of x is sum from 0 to x of x squared'
# text_sample = 'f of x equals x squared'
conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(conversion_text_sample)
print(non_finetuned_latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: f of x is sum from 0 to x of x squared
LaTeX:
LCT
English: f of x is sum from 0 to x of x squared
LaTeX: f of x is sum from 0 to x of x squared
LaTeX: f of x is


In [76]:
# print(non_finetuned_latex_generator(
#     conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
#     max_length=len(tokenizer.encode(conversion_text_sample)) + 20
# )[0]['generated_text'])

## Dataset to train

In [ ]:
data = pd.read_csv('../data/english_to_latex.csv')

print(data.shape)

data.head(2)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token



In [16]:
# Add our singular prompt
CONVERSION_PROMPT = 'LCT\n'  # LaTeX conversion task

CONVERSION_TOKEN = 'LaTeX:'

In [65]:
# This is our "training prompt" that we want GPT2 to recognize and learn
training_examples = f'{CONVERSION_PROMPT}English: ' + data['English'] + '\n' + CONVERSION_TOKEN + ' ' + data['LaTeX'].astype(str)

print(training_examples[0])

NameError: name 'data' is not defined

In [ ]:
task_df = pd.DataFrame({'text': training_examples})

task_df.head(2)

In [ ]:
latex_data = Dataset.from_pandas(task_df)  # turn a pandas DataFrame into a Dataset

def preprocess(examples):  # tokenize our text but don't pad because our collator will pad for us dynamically
    return tokenizer(examples['text'], truncation=True)

latex_data = latex_data.map(preprocess, batched=True)

## Finetune

In [ ]:
!det experiment create \
    /run/determined/workdir/gpt2-determined-finetune-poc/determined_files/const_eng_to_latex.yaml \
    /run/determined/workdir/gpt2-determined-finetune-poc/determined_files/ 

# Lets see how trained checkpoint performs

In [50]:
!det checkpoint download 6979cccf-c48e-4d3f-9543-3b0dee459f91

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Local checkpoint path:
checkpoints/6979cccf-c48e-4d3f-9543-3b0dee459f91 

   Experiment ID |   Trial ID |   Steps Completed | Report Time              | Checkpoint UUID                      | Validation Metrics                       | Metadata
-----------------+------------+-------------------+--------------------------+--------------------------------------+------------------------------------------+----------------------------------------
              27 |         27 |                37 | 2023-03-23 21:08:15+0000 | 6979cccf-c48e-4d3f-9543-3b0dee459f91 | {                                        | {
                 |            |                   |                 

In [51]:
loaded_model = GPT2LMHeadModel.from_pretrained('gpt2')
ckpt = torch.load('./checkpoints/6979cccf-c48e-4d3f-9543-3b0dee459f91/state_dict.pth')
# print(len(ckpt['models_state_dict']))
loaded_model.load_state_dict(ckpt['models_state_dict'][0])
latex_generator = pipeline('text-generation', model=loaded_model, tokenizer=tokenizer)

In [77]:
# text_sample = 'f of x equals integral from 0 to pi of x to the fourth power'
text_sample = 'f of x is sum from 0 to x of x squared'

conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(conversion_text_sample)

LCT
English: f of x is sum from 0 to x of x squared
LaTeX:


In [79]:
print(latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = x^2 \,dx^2 \,dx^2 \,


In [ ]:
#TODO

# How does Multi-GPU and Multi-Node GPU improve finetuning?